In [96]:
from PIL import Image
import torch
import torch.nn as nn
from torchvision import transforms
import joblib
import torchvision.models as models
import numpy as np
import pandas as pd
import cv2
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm

In [97]:
# Read the models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes=4):
        super(ResNetClassifier, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        # Modify the final fully connected layer to output num_classes
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

# Instantiate the model
model = ResNetClassifier(num_classes=4).to(device)
model.load_state_dict(torch.load('./models/resnet.pth', map_location=torch.device(device)))
 #
hats = joblib.load('./models/hats/hats.joblib')

#read bottom models
bottomsilhouette = joblib.load('./models/bottom/bottom_silhouette.joblib')
bottomlength = joblib.load('./models/bottom/bottom_length.joblib')
bottomclosure = joblib.load('./models/bottom/bottom_closure.joblib')
bottomknit = joblib.load('./models/bottom/bottom_knit.joblib')
bottomwaist = joblib.load('./models/bottom/bottom_waist.joblib')
bottomwoven = joblib.load('./models/bottom/bottom_woven.joblib')

#read shoes models
shoes_cane_height = joblib.load('./models/shoes/model_cane_height.joblib')
shoes_heel_shape = joblib.load('./models/shoes/model_heel_shape.joblib')
shoes_toecap = joblib.load('./models/shoes/model_toecap.joblib')

#read top models
top_silhouette = joblib.load('./models/tops/top_silhouette.joblib')
top_closure_placement = joblib.load('./models/tops/top_closure_placement.joblib')
top_knit = joblib.load('./models/tops/top_knit.joblib')
top_length = joblib.load('./models/tops/top_length.joblib')
top_neck = joblib.load('./models/tops/top_neck.joblib')
top_sleeve = joblib.load('./models/tops/top_sleeve.joblib')
top_woven = joblib.load('./models/tops/top_woven.joblib')

C:\Users\Arnau\AppData\Local\Temp\ipykernel_23228\3754075685.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./models/resnet.pth', map

In [ ]:
def generatemodels():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    class ResNetClassifier(nn.Module):
        def __init__(self, num_classes=4):
            super(ResNetClassifier, self).__init__()
            self.resnet = models.resnet18(pretrained=True)
            # Modify the final fully connected layer to output num_classes
            self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

        def forward(self, x):
            return self.resnet(x)

    # Instantiate the model
    model = ResNetClassifier(num_classes=4).to(device)
    model.load_state_dict(torch.load('./models/resnet.pth', map_location=torch.device(device)))
    #
    hats = joblib.load('./models/hats/hats.joblib')

    #read bottom models
    bottomsilhouette = joblib.load('./models/bottom/bottom_silhouette.joblib')
    bottomlength = joblib.load('./models/bottom/bottom_length.joblib')
    bottomclosure = joblib.load('./models/bottom/bottom_closure.joblib')
    bottomknit = joblib.load('./models/bottom/bottom_knit.joblib')
    bottomwaist = joblib.load('./models/bottom/bottom_waist.joblib')
    bottomwoven = joblib.load('./models/bottom/bottom_woven.joblib')

    #read shoes models
    shoes_cane_height = joblib.load('./models/shoes/model_cane_height.joblib')
    shoes_heel_shape = joblib.load('./models/shoes/model_heel_shape.joblib')
    shoes_toecap = joblib.load('./models/shoes/model_toecap.joblib')

    #read top models
    top_silhouette = joblib.load('./models/tops/top_silhouette.joblib')
    top_closure_placement = joblib.load('./models/tops/top_closure_placement.joblib')
    top_knit = joblib.load('./models/tops/top_knit.joblib')
    top_length = joblib.load('./models/tops/top_length.joblib')
    top_neck = joblib.load('./models/tops/top_neck.joblib')
    top_sleeve = joblib.load('./models/tops/top_sleeve.joblib')
    top_woven = joblib.load('./models/tops/top_woven.joblib')

    return model, hats, bottomsilhouette, bottomlength, bottomclosure, bottomknit, bottomwaist, bottomwoven, shoes_cane_height, shoes_heel_shape, shoes_toecap, top_silhouette, top_closure_placement, top_knit, top_length, top_neck, top_sleeve, top_woven

In [98]:
train_data = pd.read_csv('./data/train_data.csv')

In [99]:
listattributes = train_data.columns[:-2:]
listattributes

Index(['cod_modelo_color', 'silhouette_type', 'closure_placement',
       'heel_shape_type', 'knit_structure', 'length_type', 'neck_lapel_type',
       'cane_height_type', 'sleeve_length_type', 'toecap_type', 'waist_type',
       'woven_structure'],
      dtype='object')

In [100]:
def transform_image(image):
    transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
    return transform(image).unsqueeze(0)

gtm = {0: 'Top', 1: 'Bottom', 2: 'Shoes', 3:'Hats'}

def classify(img):
    #transform img
    image = Image.open(img)
    image = transform_image(image)
    image = image.to(device)
    model.eval()
    with torch.no_grad():
        output = model(image)
    return gtm[torch.argmax(output).item()]
    

In [101]:
image='./images/images/81_1072506_77097613-09_B.jpg'

In [102]:
def attHat(img):
    img = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (64, 64))
    img_flat = img.flatten()
    pred = hats.predict([img_flat])[0].replace('-1', 'INVALID')
    return ('silhouette_type',pred)


In [103]:
def attShoes(img):
    img = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (64, 64))
    img_flat = img.flatten()
    pred1 = shoes_cane_height.predict([img_flat])[0].replace('-1', 'INVALID')
    pred2 = shoes_heel_shape.predict([img_flat])[0].replace('-1', 'INVALID')
    pred3 = shoes_toecap.predict([img_flat])[0].replace('-1', 'INVALID')
    return ('cane_height_type',pred1), ('heel_shape_type',pred2), ('toecap_type',pred3)

In [104]:
def attBottom(img):
    img = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (64, 64))
    img_flat = img.flatten()
    pred1 = bottomsilhouette.predict([img_flat])[0].replace('-1', 'INVALID')
    pred2 = bottomlength.predict([img_flat])[0].replace('-1', 'INVALID')
    pred3 = bottomclosure.predict([img_flat])[0].replace('-1', 'INVALID')
    pred4 = bottomknit.predict([img_flat])[0].replace('-1', 'INVALID')
    pred5 = bottomwaist.predict([img_flat])[0].replace('-1', 'INVALID')
    pred6 = bottomwoven.predict([img_flat])[0].replace('-1', 'INVALID')
    return ('silhouette_type',pred1), ('length_type',pred2), ('closure_placement',pred3), ('knit_structure',pred4), ('waist_type',pred5), ('woven_structure',pred6)

In [105]:
def attTop(img):
    img = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (64, 64))
    img_flat = img.flatten()
    pred1 = top_silhouette.predict([img_flat])[0].replace('-1', 'INVALID')
    pred2 = top_closure_placement.predict([img_flat])[0].replace('-1', 'INVALID')
    pred3 = top_knit.predict([img_flat])[0].replace('-1', 'INVALID')
    pred4 = top_length.predict([img_flat])[0].replace('-1', 'INVALID')
    pred5 = top_neck.predict([img_flat])[0].replace('-1', 'INVALID')
    pred6 = top_sleeve.predict([img_flat])[0].replace('-1', 'INVALID')
    pred7 = top_woven.predict([img_flat])[0].replace('-1', 'INVALID')
    return ('silhouette_type',pred1), ('closure_placement',pred2), ('knit_structure',pred3), ('length_type',pred4), ('neck_lapel_type',pred5), ('sleeve_length_type',pred6), ('woven_structure',pred7)


In [106]:
#load test data
test_data = pd.read_csv('./data/test_data.csv')

In [107]:
files=set()
images=[]
for i, row in test_data.iterrows():
    if row['des_filename'] in files:
        continue
    files.add(row['des_filename'])
    images.append((row['des_filename'], row['cod_modelo_color']))

In [108]:
listattributes

Index(['cod_modelo_color', 'silhouette_type', 'closure_placement',
       'heel_shape_type', 'knit_structure', 'length_type', 'neck_lapel_type',
       'cane_height_type', 'sleeve_length_type', 'toecap_type', 'waist_type',
       'woven_structure'],
      dtype='object')

In [119]:
attributes = set(listattributes) - {'cod_modelo_color'}
len(attributes)

11

In [124]:
submission = []
for img,cod in tqdm(images, desc='pred'):
    if img != '88_49720742_67044470-56_.jpg':
        image = f'./images/images/{img}'
        attval = {key: 'INVALID' for key in attributes}    
        if classify(image) == 'Hats':
            att = attHat(image)
            code = cod+'_'+at[0]
            attval[at[0]] = at[1]
        elif classify(image) == 'Shoes':
            att = attShoes(image)
            for at,val in att:
                code = cod+'_'+at
                attval[at] = val
        elif classify(image) == 'Bottom':
            att = attBottom(image)
            for at,val in att:
                code = cod+'_'+at
                attval[at] = val
        else:
            att = attTop(image)
            for at,val in att:
                code = cod+'_'+at
                attval[at] = val
    for k in attval.keys():
        submission.append((cod+'_'+k, attval[k]))

pred: 100%|██████████| 6529/6529 [06:52<00:00, 15.82it/s]


In [125]:
len(submission)

71858

In [126]:
submission

[('88_49711373_knit_structure', 'INVALID'),
 ('88_49711373_closure_placement', 'INVALID'),
 ('88_49711373_cane_height_type', 'Alta'),
 ('88_49711373_length_type', 'INVALID'),
 ('88_49711373_sleeve_length_type', 'INVALID'),
 ('88_49711373_woven_structure', 'INVALID'),
 ('88_49711373_neck_lapel_type', 'INVALID'),
 ('88_49711373_silhouette_type', 'INVALID'),
 ('88_49711373_toecap_type', 'Con punta'),
 ('88_49711373_waist_type', 'INVALID'),
 ('88_49711373_heel_shape_type', 'Plano'),
 ('88_49718802_knit_structure', 'INVALID'),
 ('88_49718802_closure_placement', 'INVALID'),
 ('88_49718802_cane_height_type', 'INVALID'),
 ('88_49718802_length_type', 'INVALID'),
 ('88_49718802_sleeve_length_type', 'INVALID'),
 ('88_49718802_woven_structure', 'INVALID'),
 ('88_49718802_neck_lapel_type', 'INVALID'),
 ('88_49718802_silhouette_type', 'INVALID'),
 ('88_49718802_toecap_type', 'Redonda'),
 ('88_49718802_waist_type', 'INVALID'),
 ('88_49718802_heel_shape_type', 'Plano'),
 ('88_49709572_knit_structure',

In [135]:
#read /data/submission_sample.csv
submission_sample = pd.read_csv('./data/sample_submission.csv')
submission_sample['des_value'] = 'INVALID'
submission_sample


,test_id,des_value
0,88_49726492_silhouette_type,INVALID
1,88_49717057_silhouette_type,INVALID
2,88_49727807_silhouette_type,INVALID
3,88_49718986_silhouette_type,INVALID
4,88_49716369_silhouette_type,INVALID
...,...,...
71814,88_49727621_toecap_type,INVALID
71815,88_49727736_toecap_type,INVALID
71816,88_49734138_toecap_type,INVALID
71817,88_49717732_toecap_type,INVALID


In [136]:
for cod, val in submission.itertuples(index=False):
    submission_sample.loc[submission_sample['test_id'] == cod, 'des_value'] = val
submission_sample

,test_id,des_value
0,88_49726492_silhouette_type,Recto
1,88_49717057_silhouette_type,Recto
2,88_49727807_silhouette_type,Recto
3,88_49718986_silhouette_type,Recto
4,88_49716369_silhouette_type,Recto
...,...,...
71814,88_49727621_toecap_type,INVALID
71815,88_49727736_toecap_type,INVALID
71816,88_49734138_toecap_type,INVALID
71817,88_49717732_toecap_type,INVALID


In [137]:
submission_sample.to_csv('submission.csv', index=False)